In [3]:
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
import scipy.linalg as sla

In [ ]:
# define lattice and generate hopping matrix with OBC

In [2]:
from pythtb import *
def Tmatrix(t,mu):

    # set model parameters
    mu=0.0
    t=-1.0
    lat=[[1.0,0.0],[0.0,1.0]]
    orb=[[0.0,0.0]]

    # create TB model
    my_model=tb_model(2,2,lat,orb)
    
    # set on-site energies
    my_model.set_onsite([-mu])
    # set hopping terms (one for each nearest neighbor)
    my_model.set_hop(t, 0, 0, [1, 0])
    my_model.set_hop(t, 0, 0, [0, 1])

    # create a 4x4 lattice
    Nx, Ny = 4, 4
    my_ham = my_model.cut_piece(Nx,0,glue_edgs=False)
    my_ham = my_ham.cut_piece(Ny,1,glue_edgs=False)
    # print Hamiltonian
    return my_ham._gen_ham()


In [44]:
L = 4 * 4
Ntau = 20
dtau = 0.1
U = 4.0
t = -1.0
mu = U/2
beta = dtau * Ntau
alpha = np.arccosh(np.exp(0.5*dtau*U))
T = Tmatrix(t,mu).real
ExpT = sla.expm(-dtau * T)
Vdiag = np.zeros(L)
Vdiag[:] = alpha

In [22]:
def ExpM(phi,ExpT,Vdiag):
    # T = exp(-T*dtau)
    return ExpT.dot(np.diag(phi * Vdiag))

In [ ]:
def SVD_stablizer_back(A,U,D,V):
    # B = U * D * V
    # B * A = U * D * V * B = U * U1 * D1 * V1 = U2 * D1 * V1
    U1, D, V = LA.svd(np.diag(D).dot(V).dot(A))
    return U.dot(U1), D, V
    
def SVD_stablizer_forward(A,U,D,V):
    # B = U * D * V
    # A * B = A * U * D * V = U1 * D1 * V1 * V = U1 * D1 * V2
    U, D, V1 = LA.svd(A.dot(U).dot(np.diag(D)))
    return U, D, V1.dot(V)

In [12]:
def GF(UL,DL,VL,UR,DR,VR):
    # B(t,0) = UL * DL * VL
    # B(beta,t) = VR * DR * UR Take care of it !!!
    U,D,V = LA.svd(LA.inv(UL.dot(UR))+np.diag(DR).dot(VR).dot(VL).dot(np.diag(DL)))
    return LA.inv(V.dot(UL)).dot(np.diag(1/D)).dot(UR.dot(U))

In [39]:
def Stable_Bmatrix(phi,ExpT,Vdiag):
    # use SVD algorithm to calculate the stable B matrix
    # ExpT: the matrix of exp(-dtau * T)
    # Vdiag: Ntau x L matrix with each column being - alpha 
    Ntau, L = phi.shape
    U = np.zeros((Ntau+1,L,L))
    D = np.zeros((Ntau+1,L))
    V = np.zeros((Ntau+1,L,L))

    U[0],D[0],V[0] = LA.svd(np.eye(L))
    for i in range(Ntau):
        U[i+1], D[i+1], V[i+1] = SVD_stablizer_forward(ExpM(phi[i], ExpT, Vdiag),U[-1],D[-1],V[-1])
    return U,D,V

In [47]:
phi = np.random.choice([1, -1], size=(Ntau, L))
#part of spin up and spin down are not coupled but direct product
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)

In [51]:
def Update_HS(GFup, GFdn, phi):
    for i in range(phi):
        dup = np.exp(2 * alpha * phi[i]) -1 
        ddn = np.exp(-2 * alpha * phi[i]) -1
        R = (1 + dup * (1 - GFup[i,i]))*(1 + ddn * (1 - GFdn[i,i]))
        
        if  np.abs(R) > np.random.rand():
            phi[i] = -phi[i]
            GFup -= dup * np.outer(GFup[:,i], 1-GFup[i,:])/R
            GFdn -= ddn * np.outer(GFdn[:,i], 1-GFdn[i,:])/R
        
    return GFup, GFdn, phi

In [ ]:
def Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable):
    # Input GFup, GFdn should be G(0,0) = G(\beta,\beta)
    # The first member of U, D, V should be the result of I = B(0,0) or B(\beta, \beta)

    Ntau = len(phi) 
    for i in range(1,Ntau+1):
        if np.mod(i,Nstable):
            # recompute equal time GF to avoid accumulation of numerical error
            GFup_recomputed = GF(Uup[i],Dup[i],Vup[i],Uup[i-1],Dup[i-1],Vup[i-1])
            GFdn_recomputed = GF(Udn[i],Ddn[i],Vdn[i],Udn[i-1],Ddn[i-1],Vdn[i-1])
            # compare with advanced one to get the accumulated error
            GFup_error = np.max(np.abs(GFup_recomputed - GFup))
            GFdn_error = np.max(np.abs(GFdn_recomputed - GFdn))
            print("Error of Green function: ", (GFup_error + GFdn_error)/2)

            GFup = GFup_recomputed
            GFdn = GFdn_recomputed
        
        # Update H-S field and Green function
        GFup, GFdn, phi[i] = Update_HS(GFup, GFdn, phi[i])

        # Update SVD decompostion for B matrix
        Bup = ExpM(phi[i], ExpT, Vdiag)
        Bdn = ExpM(phi[i], ExpT, -Vdiag)

        # Advance equal time GF using new H-S field
        GFup = Bup.dot(GFup).dto(LA.inv(Bup))
        GFdn = Bdn.dot(GFup).dto(LA.inv(Bdn))

        # Repalce SVD decomposition of B matrix
        Uup[i], Dup[i], Vup[i] = SVD_stablizer_back(Bup, Uup[i], Dup[i], Vup[i])
        Udn[i], Ddn[i], Vdn[i] = SVD_stablizer_back(Bdn, Udn[i], Ddn[i], Vdn[i])

    return GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi


